In [29]:
from PIL import Image
import numpy as np
from scipy.ndimage import map_coordinates
import matplotlib.pyplot as plt

kiwi = Image.open("kiwi.jpg").convert("RGB")
arr = np.array(kiwi)

translated = np.zeros_like(arr)
translated[30:, 50:] = arr[:-30, :-50]

def apply_wave_effect(image):
    rows, cols, ch = image.shape
    coords_x, coords_y = np.meshgrid(np.arange(cols), np.arange(rows))
    coords_x = coords_x + 10 * np.sin(2 * np.pi * coords_y / 60)
    coords = np.array([coords_y, coords_x])
    warped = np.zeros_like(image)
    for c in range(3):
        warped[..., c] = map_coordinates(image[..., c], coords, order=1, mode='reflect')
    return warped

wave_img = apply_wave_effect(translated)

Image.fromarray(wave_img).save("kiwi_wave.jpg")


In [30]:
from PIL import Image
import numpy as np

papaya = Image.open("papaya.png").convert("RGBA")
watermelon = Image.open("watermelon.png").convert("RGBA")

def apply_gradient(image, color1, color2):
    arr = np.array(image)
    h, w = arr.shape[:2]
    gradient = np.linspace(np.array(color1), np.array(color2), h).astype(np.uint8)
    for y in range(h):
        arr[y, :, :3] = gradient[y]
    return Image.fromarray(arr)

papaya_grad = apply_gradient(papaya, [255, 0, 0], [0, 255, 0])         # đỏ -> xanh
watermelon_grad = apply_gradient(watermelon, [255, 255, 0], [128, 0, 128])  # vàng -> tím

canvas = Image.new("RGBA", (max(papaya_grad.width, watermelon_grad.width) * 2, 
                            max(papaya_grad.height, watermelon_grad.height)), (0,0,0,0))

canvas.paste(papaya_grad, (0, 0), papaya_grad)
canvas.paste(watermelon_grad, (papaya_grad.width, 0), watermelon_grad)

canvas.save("fruit_gradient.png")


In [31]:
from PIL import ImageOps

mountain = Image.open("mountain.jpg").convert("RGB")
boat = Image.open("boat.jpg").convert("RGB")

mountain_rot = mountain.rotate(45, expand=False)
boat_rot = boat.rotate(45, expand=False)

mountain_flip = ImageOps.mirror(mountain_rot)
boat_flip = ImageOps.mirror(boat_rot)

canvas = Image.new("RGB", (mountain_flip.width + boat_flip.width, max(mountain_flip.height, boat_flip.height)), "white")
canvas.paste(mountain_flip, (0, 0))
canvas.paste(boat_flip, (mountain_flip.width, 0))

canvas.save("mountain_boat_mirror.jpg")


In [32]:
from PIL import Image
import numpy as np
from scipy.ndimage import geometric_transform

pagoda = Image.open("pagoda.jpg").convert("RGB")
pagoda = pagoda.resize((pagoda.width * 5, pagoda.height * 5))

arr = np.array(pagoda)

def warp(coord):
    y, x = coord
    shift = 20.0 * np.sin(2 * np.pi * x / 200)
    return y + shift, x

warped_channels = []
for c in range(3):
    warped = geometric_transform(arr[..., c], warp, order=1, mode='reflect')
    warped_channels.append(warped)

warped_image = np.stack(warped_channels, axis=2).astype(np.uint8)

Image.fromarray(warped_image).save("pagoda_warped.jpg")



In [ ]:
import cv2

def menu_transform(image_path):
    image = cv2.imread(image_path)

    while True:
        print("\nMENU:")
        print("1. Tịnh tiến")
        print("2. Xoay")
        print("3. Zoom")
        print("4. Gaussian Blur")
        print("5. Sóng")
        print("0. Thoát")
        choice = input("Chọn thao tác: ")

        if choice == '1':
            dx = int(input("Dịch ngang: "))
            dy = int(input("Dịch dọc: "))
            M = np.float32([[1, 0, dx], [0, 1, dy]])
            image = cv2.warpAffine(image, M, (image.shape[1], image.shape[0]))

        elif choice == '2':
            angle = float(input("Góc xoay: "))
            reshape = input("Giữ kích thước (y/n): ").lower() != 'y'
            h, w = image.shape[:2]
            M = cv2.getRotationMatrix2D((w//2, h//2), angle, 1)
            if reshape:
                image = cv2.warpAffine(image, M, (int(1.5*w), int(1.5*h)))
            else:
                image = cv2.warpAffine(image, M, (w, h))

        elif choice == '3':
            scale = float(input("Hệ số zoom: "))
            image = cv2.resize(image, None, fx=scale, fy=scale)

        elif choice == '4':
            sigma = float(input("Sigma blur: "))
            ksize = int(2 * round(sigma * 3) + 1)
            image = cv2.GaussianBlur(image, (ksize, ksize), sigma)

        elif choice == '5':
            rows, cols = image.shape[:2]
            map_x = np.zeros((rows, cols), dtype=np.float32)
            map_y = np.zeros((rows, cols), dtype=np.float32)
            amplitude = float(input("Biên độ sóng: "))
            for i in range(rows):
                for j in range(cols):
                    map_x[i, j] = j
                    map_y[i, j] = i + amplitude * np.sin(2 * np.pi * j / 100)
            image = cv2.remap(image, map_x, map_y, cv2.INTER_LINEAR)

        elif choice == '0':
            break

        else:
            print("Sai lựa chọn")

        cv2.imshow("Kết quả", image)
        cv2.waitKey(0)

    cv2.imwrite("output_final.jpg", image)
    cv2.destroyAllWindows()

menu_transform("input.jpg")



MENU:
1. Tịnh tiến
2. Xoay
3. Zoom
4. Gaussian Blur
5. Sóng
0. Thoát
